# 🔍 Policy Checker Agent with Azure AI Search

This notebook demonstrates how to create an intelligent policy checker agent using Azure AI Agents Service integrated with Azure AI Search. The agent specializes in analyzing auto insurance policies and validating claim coverage by searching through policy documents.

## What You'll Learn:
- 🤖 How to create Azure AI Agents with search capabilities
- 🔎 Integrating Azure AI Search with AI agents
- 📋 Building policy analysis and coverage validation logic
- 🛡️ Implementing insurance policy checking workflows
- 💼 Working with different policy types (auto, motorcycle, commercial)


## Setup and Imports

In [ ]:
# Import necessary libraries and modules
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, AzureCliCredential, ChainedTokenCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole

# Load environment variables from .env file
load_dotenv(dotenv_path='../../.env', override=True)  # This forces a reload of the .env file

print("✅ Libraries imported and environment variables loaded")

print("\n🔍 Setting up Azure authentication...")

# Debug: Check if critical environment variables are loaded

required_vars = ['AI_FOUNDRY_PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME', 'AZURE_AI_CONNECTION_ID']        print(f"❌ {var}: Not found")

for var in required_vars:    else:

    value = os.environ.get(var)        print(f"✅ {var}: Found")
    if value:

## Configuration

In [ ]:
# Retrieve endpoint and model deployment name from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")  # Ensure the PROJECT_ENDPOINT environment variable is set
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")  # Ensure the MODEL_DEPLOYMENT_NAME environment variable is set
sc_connection_id = os.environ.get("AZURE_AI_CONNECTION_ID")  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set

# Enhanced Azure authentication with fallback options
print("🔐 Attempting Azure authentication...")

# Try multiple authentication methods in order of preference
try:
    # First, try Azure CLI credentials (if you're signed in with az login)
    cli_credential = AzureCliCredential()
    
    # Create a chained credential that falls back through multiple options
    credential = ChainedTokenCredential(
        cli_credential,  # Try Azure CLI first
        DefaultAzureCredential(exclude_interactive_browser_credential=False)  # Then default options
    )
    
    print("✅ Authentication credential created successfully")
    
except Exception as e:
    print(f"❌ Authentication setup failed: {e}")
    print("\n💡 To fix this:")
    print("1. Run 'az login' in your terminal")
    print("2. Or sign in to Azure through VS Code (Azure extension)")
    raise

# Verify we have all required environment variables
if not all([project_endpoint, model_deployment_name, sc_connection_id]):
    missing = []
    if not project_endpoint: missing.append("AI_FOUNDRY_PROJECT_ENDPOINT")
    if not model_deployment_name: missing.append("MODEL_DEPLOYMENT_NAME")
    if not sc_connection_id: missing.append("AZURE_AI_CONNECTION_ID")
    
    print(f"❌ Missing required environment variables: {missing}")
    print("Please check your .env file and ensure these variables are set.")
    raise ValueError(f"Missing environment variables: {missing}")

print(f"✅ All required environment variables loaded")
print(f"📍 Project endpoint: {project_endpoint[:50]}...")

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=credential,  # Use our enhanced credential
)

print("✅ AIProjectClient created successfully")

with project_client:
    # Initialize the Azure AI Search tool with the required parameters
    ai_search = AzureAISearchTool(
        index_connection_id=sc_connection_id,  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set
        index_name="insurance-documents-index",  # Name of the search index
        query_type=AzureAISearchQueryType.SIMPLE,  # Query type (e.g., SIMPLE, FULL)
        top_k=3,  # Number of top results to retrieve
        filter="",  # Optional filter for search results
    )

    # Create an agent with the specified model, name, instructions, and tools
    agent = project_client.agents.create_agent(
        model=model_deployment_name,  # Model deployment name
        name="policy-checker",  # Name of the agent
        instructions=
        """"
        You are an expert Insurance Policy Checker Agent specialized in analyzing auto insurance policies and validating claim coverage. Your primary responsibilities include:

        **Core Functions:**
        - Analyze insurance policy documents to determine coverage details
        - Validate if specific claims are covered under policy terms
        - Explain policy limits, deductibles, and exclusions
        - Identify coverage gaps or restrictions
        - Provide clear explanations of policy benefits

        **Policy Types You Handle:**
        - Commercial Auto Policies
        - Comprehensive Auto Policies  
        - High Value Vehicle Policies
        - Liability Only Policies
        - Motorcycle Policies

        **Analysis Guidelines:**
        1. Always reference specific policy sections when providing coverage determinations
        2. Clearly state coverage limits, deductibles, and any applicable restrictions
        3. Identify any exclusions that may apply to the claim
        4. Be precise about effective dates and policy periods
        5. Flag any discrepancies between claim details and policy terms

        **Response Format:**
        - Start with a clear coverage determination (COVERED/NOT COVERED/PARTIAL COVERAGE)
        - Provide the specific policy section reference
        - Explain coverage limits and deductibles
        - List any relevant exclusions or conditions
        - Suggest next steps if coverage issues exist
        - Everything in a clear, concise manner in one paragraph.

        **Tone:** Professional, accurate, and helpful. Always be thorough in your analysis while remaining clear and concise.

                    break  # Only show the first agent response

        When you cannot find specific information in the policy documents, clearly state what information is missing and what additional documentation would be needed.                    print(agent_response)

        """,  # Instructions for the agent                    agent_response = content_item['text']['value']

        tools=ai_search.definitions,  # Tools available to the agent                if content_item.get('type') == 'text' and 'text' in content_item:

        tool_resources=ai_search.resources,  # Resources for the tools                content_item = message.content[0]

    )            if message.content and len(message.content) > 0:

    print(f"Created agent, ID: {agent.id}")        if message.role == MessageRole.AGENT:

        # Only show the agent's response, not the user's question

    # Create a thread for communication with the agent    for message in messages:

    thread = project_client.agents.threads.create()    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    print(f"Created thread, ID: {thread.id}")    # Fetch and log all messages from the thread



    # Send a message to the thread        print(f"Run failed: {run.last_error}")

    message = project_client.agents.messages.create(        # Log the error if the run fails

        thread_id=thread.id,  # ID of the thread    if run.status == "failed":

        role=MessageRole.USER,  # Role of the message sender (e.g., user)

        content="What motorcycle coverage options are available?",  # Message content    print(f"Run finished with status: {run.status}")

    )    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

    print(f"Created message, ID: {message['id']}")    # Create and process an agent run in the thread using the tools


Amazing! So what you have done so far is creating an agent, that consults the Azure AI Search Index that searches on your policies databases. Your query has made it such as it replies to your query with a text based solution. Let's print it out!